# Annotations: To YOLOV5 Format

In [69]:
import json
import yaml
from bidict import bidict
from PIL import Image

## Configs

In [70]:
labels = bidict({"Front": 0, "Side": 1, "Back": 2})
img_dir = os.path.join("../Processing/Dataset_v1")
input_json = os.path.join("./yolov5_1/", "YoloAnnotationV2.json")
output_dir = os.path.join("./yolov5_1/", "YoloAnnotation/")

## Prep. Directories

In [71]:
assert len(os.listdir(img_dir)) > 0

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Get Formatted Annotations

To check correctness, make sure that for image car99.jpg, values are close to these: 

0 0.6430288461538461 0.39302884615384615 0.7115384615384616 0.78125

2 0.19230769230769232 0.3076923076923077 0.18509615384615385 0.6057692307692307

In [72]:
annotations1 = json.load(open(input_json))
annotations = list(annotations1.values())[1]

In [73]:
missedAnnotation = 0
for item in annotations.keys(): 
    element = annotations[item]

    regions = element["regions"]
    filename = element["filename"]

    # Needed to normalise coordinates
    im = Image.open(os.path.join(img_dir, filename))
    img_width, img_height = im.size

    yolo_annotations = []    
    print("Img Name: ", filename)
    print("-----------------------------------")
    for region in regions: 
        if region["region_attributes"]:
            print("Region: ", region["shape_attributes"]) # Properties: x, y, width, height
            class_name = region["region_attributes"]["Car Side"]

            # From Format: x, y, w, h measured from top left coordinates
            x = region["shape_attributes"]["x"]
            y = region["shape_attributes"]["y"]
            w = region["shape_attributes"]["width"]
            h = region["shape_attributes"]["height"]

            # To Format: x, y, w, h measured from center coordinates
            x_center = x + w/2
            y_center = y + h/2

            ## Normalizing for YOLO Format: 
            print(x)
            yolo_format = [ labels[class_name], x_center / img_width, y_center / img_height, w / img_width, h / img_height ]

            print("Class Label: ", labels[class_name])
            print("Yolo Annotation: ", yolo_format)
            yolo_annotations.append(yolo_format)
        else: 
            missedAnnotation +=1
    
    # Saving to txt
    yolo_annotations = [" ".join([str(e) for e in yolo_format]) for yolo_format in yolo_annotations]
    with open(os.path.join(output_dir, filename.split('.')[0]+'.txt'), 'w') as f:
        f.writelines("\n".join(yolo_annotations))
    print("-----------------------------------")
print(missedAnnotation)

Img Name:  car99.jpg
-----------------------------------
Region:  {'name': 'rect', 'x': 294, 'y': 2, 'width': 728, 'height': 800}
294
Class Label:  2
Yolo Annotation:  [2, 0.642578125, 0.392578125, 0.7109375, 0.78125]
Region:  {'name': 'rect', 'x': 103, 'y': 5, 'width': 189, 'height': 620}
103
Class Label:  1
Yolo Annotation:  [1, 0.19287109375, 0.3076171875, 0.1845703125, 0.60546875]
-----------------------------------
Img Name:  car98.jpg
-----------------------------------
Region:  {'name': 'rect', 'x': 51, 'y': 279, 'width': 634, 'height': 611}
51
Class Label:  0
Yolo Annotation:  [0, 0.359375, 0.57080078125, 0.619140625, 0.5966796875]
Region:  {'name': 'rect', 'x': 477, 'y': 301, 'width': 470, 'height': 372}
477
Class Label:  1
Yolo Annotation:  [1, 0.6953125, 0.4755859375, 0.458984375, 0.36328125]
Region:  {'name': 'rect', 'x': 305, 'y': 195, 'width': 113, 'height': 89}
305
Class Label:  0
Yolo Annotation:  [0, 0.35302734375, 0.23388671875, 0.1103515625, 0.0869140625]
-----------

# Get YAML file for YOLOV5

In [79]:
# train: data/YoloDataset/train/images
# val: data/YoloDataset/test/images

# nc: 3
# names: ['Back', 'Front', 'Side']
data_file = [   "train: <TO BE FILLED LATER>",
                "val: <TO BE FILLED LATER>",
                "test: <OPTIONAL: TO BE FILLED LATER>",
                "",
                "nc: " + str(len(labels)), 
                "names: " + str([labels.inverse[i] for i in range(len(labels))])
            ]
print(data_file)

['train: <TO BE FILLED LATER>', 'val: <TO BE FILLED LATER>', 'test: <OPTIONAL: TO BE FILLED LATER>', '', 'nc: 3', "names: ['Front', 'Side', 'Back']"]


In [77]:
# Saving to txt
with open(os.path.join(output_dir, 'data.yaml'), 'w') as f:
    f.writelines("\n".join(data_file))